## Create Index

In [8]:
%load_ext autoreload
%autoreload 2

In [1]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index import (
    TreeIndex,
    VectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    ServiceContext,
    Response,
)
from llama_index.llms import OpenAI
from llama_index.evaluation import ResponseEvaluator
import pandas as pd

In [3]:
# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

evaluator_gpt4 = ResponseEvaluator(service_context=service_context_gpt4)

In [5]:
documents = SimpleDirectoryReader("./llamaindex_data/").load_data()
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query("How did New York City get its name?")

In [7]:
print(response_vector)


New York City was named in honor of the Duke of York, who would become King James II of England. In 1664, King Charles II appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control. The city was then renamed New York in his honor.


In [10]:
from ragas.llama_index import evaluate

In [11]:
from llama_index.evaluation import DatasetGenerator

question_generator = DatasetGenerator.from_documents(documents)
eval_questions = question_generator.generate_questions_from_nodes(5)

eval_questions

['What is the population of New York City as of 2020?',
 'Which city is the second-largest in the United States after New York City?',
 'What is the geographical and demographic center of the New York metropolitan area?',
 'How many people live within 250 miles of New York City?',
 'What is the largest metropolitan economy in the world as of 2021?']

In [19]:
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy
from ragas.metrics.critique import harmfulness

metrics = [faithfulness, answer_relevancy, context_relevancy, harmfulness]

In [ ]:
r = evaluate(query_engine, metrics, eval_questions)

> /home/jjmachan/jjmachan/explodinggradients/ragas/src/ragas/validation.py(51)validate_evaluation_modes()
     50         import ipdb; ipdb.set_trace()  # fmt: skip
---> 51         if required_columns.symmetric_difference(available_columns):
     52             raise ValueError(

ipdb> required_columns
{'question', 'contexts', 'answer'}
ipdb> available_columns
{'question', 'contexts', 'answer'}
ipdb> required_columns.issubset(available_columns)
True
ipdb> required_columns.symmetric_difference(available_columns)
set()
ipdb> c
> /home/jjmachan/jjmachan/explodinggradients/ragas/src/ragas/validation.py(51)validate_evaluation_modes()
     50         import ipdb; ipdb.set_trace()  # fmt: skip
---> 51         if required_columns.symmetric_difference(available_columns):
     52             raise ValueError(

ipdb> required_columns.issubset(available_columns)
True
ipdb> required_columns
{'question', 'answer'}
ipdb> required_columns.symmetric_difference(available_col
*** SyntaxError: '(' was nev

In [ ]:
r

In [ ]:
r.to_pandas()